In [1]:
def DF_tree_traversal(Binary_Tree, order='preorder'):
    '''Inplementation of Depth-first tree traversal
    
    Args:
    - Binary_Tree: binary tree
    - order: traversal order, default 'preorder'
    
    '''
    if order not in ['preorder', 'inorder', 'postorder']:
        raise Exception("'order' can noly be 'preorder', 'inorder' or 'postorder'")
        
    def preorder(node, indent='-'):
        if node is None:
            return None   
        print(indent+str(node.value))
        indent = '|'+indent
        preorder(node.left, indent=indent)
        preorder(node.right, indent=indent)
        
    def inorder(node):
        if node is None:
            return None
        inorder(node.left)
        print(node.value)
        inorder(node.right)
        
    def postorder(node):
        if node is None:
            return None
        postorder(node.left)
        postorder(node.right)
        print(node.value)
        
    if order == 'preorder':
        preorder(Binary_Tree.root)
    elif order == 'inorder':
        inorder(Binary_Tree.root)
    else:
        postorder(Binary_Tree.root)

In [2]:
class Bi_node:
    '''Implementation of binary search tree node
    
    Args:
    - value: int, the value of binary search tree node
    
    Attributes:
    - value: int, the value of binary search tree node
    - left: Bi_node, left node of this node, default None
    - right: Bi_node, right node of this node, default None
    - prev: Bi_node, parent node of this node, default None
    
    '''
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None
        self.prev = None

        
class Bi_Search_Tree:
    '''Implementation of binary search tree
        
    Attributes:
    - root: Bi_node, the root node of binary search tree
    
    '''
    
    def __init__(self):
        self.root = None
        
    
    def search(self, value):
        # O(h)
        '''Determine whether the value is in the tree
        
        Return: 
        - Bi_node, if the node of value is found, otherwise False
        
        '''
        temp = self.root
        while temp is not None:
            if value == temp.value:
                return temp
            elif value < temp.value:
                temp = temp.left
            else:
                temp = temp.right
        return False
            
    
    def insert(self, value):
        # O(h)
        '''Insert a new node at the bottom of the tree
        
        Args:
        - value: int, the value of new node
        
        Return: 
        - None
        
        '''
        temp = self.root
        
        # 1. find a place for inserting new node
        while temp is not None: 
            prev = temp                  # find the parent node of new node
            if value < temp.value:       # go left if smaller  
                temp = temp.left        
            elif value > temp.value:     # go right if bigger
                temp =temp.right        
            else:                        # do not allow duplicate value
                return None
        
        # 2. insert new value into leaf node
        if self.root is None:            # if tree is empty
            self.root = Bi_node(value)   # -> make new node as the root node
        elif value < prev.value:         # if new value is smaller than leaf node
            prev.left = Bi_node(value)   # -> make new node as the left node of leaf node
            prev.left.prev = prev        # -> assign new node's parent node
        else:                            # if new value is bigger than leaf node
            prev.right = Bi_node(value)  # -> make new node as the right node of leaf node
            prev.right.prev = prev       # -> assign new node's parent node
            
            
    def minimum(self, subtree_value=None):
        '''Find the minimal value of the tree
        
        Args:
        - subtree_value: int, value of subtree root node, default None(the whole tree)
        
        Return: 
        - Bi_node, the minimal node of the tree(subtree)
        
        '''
        if subtree_value is None:             # search the whole tree
            temp = self.root
        else: 
            temp = self.search(subtree_value) # find the subtree
            
        # go left all the way
        while temp.left is not None:
            temp = temp.left
        return temp
    
    
    def successor(self, value):
        '''Find the successor node of the value
        
        Args:
        - value: int, the value of the node
        
        Return: 
        - Bi_node, return the successor node if found, otherwise None
        
        '''
        
        # 1. find the node of value
        temp = self.search(value)
        if temp is False:
            raise Exception("The value is not in the tree")
                
        # 2. find the successor
        # 2.1 if the node have right subtree
        #     -> successor is the minimum of right subtree
        if temp.right is not None:           
            return self.minimum(temp.right.value)
        
        # 2.2 if the node don't have right subtree
        while (temp.prev is not None) and (temp == temp.prev.right):
            temp = temp.prev
        return temp.prev
        
        
    def delete(self, value):
        '''Delete the node of the value
        
        Args:
        - value: int, the value of the node
        
        Return:
        - None
        
        '''
        # 1. find the node of value
        temp = self.search(value)
        if temp is False:
            raise Exception("The value is not in the tree")
            
        # 2. delelte the node
        # 2.1 if the node has no child, simply delete the node
        if (temp.left==None) and (temp.right==None):
            if temp is self.root:             # if there is only one root node in the tree
                self.root = None              # delete the root node
                return None
            if temp.prev.left == temp:
                temp.prev.left = None
            else:
                temp.prev.right = None   
                           
                    
        # 2.2 if the node has one child, replace it by it's child
        elif (temp.left==None) or (temp.right==None):
            child = temp.left if temp.left is not None else temp.right
            child.prev = temp.prev
            if temp is not self.root:
                if temp==temp.prev.left:
                    temp.prev.left = child 
                else:
                    temp.prev.right = child
            else:
                self.root = child
            
            
        # 2.3 if the node has two children, replace it by it's successor
        else:
            # 2.3.1 find the successor
            successor = self.successor(value)
            
            # special case: successor.prev is temp
            # (to be more specify, successor is the right child of the temp)
            if successor.prev is temp:
                successor.prev = temp.prev
                successor.left = temp.left
                if temp is not self.root:
                    if temp.prev.left == temp:
                        temp.prev.left = successor
                    else:
                        temp.prev.right = successor
                else:
                    self.root = successor
                    
            else:
                # 2.3.2 take the successor out of the tree
                # successor can only have a right child node
                if successor.right is not None:                
                    successor.right.prev = successor.prev  
                # successor must on it's parent's left side
                successor.prev.left = successor.right  
                
                # 2.3.3 replace the node by the successor
                # connect successor with temp's left node
                successor.left = temp.left
                temp.left.prev = successor
                # connect successor with temp's right node
                successor.right = temp.right
                temp.right.prev = successor
                # connect successor with temp's parent node
                if temp is not self.root:
                    if temp.prev.left == temp:
                        temp.prev.left = successor
                    else:
                        temp.prev.right = successor
                else:
                    self.root = successor

In [3]:
# Create an binary search tree
BST = Bi_Search_Tree()
for i in [15, 9, 18, 7, 13, 16, 19, 8]:
    BST.insert(i)

# Preorder deep-first tree traversal
print('Original tree:')
DF_tree_traversal(BST, order='preorder')

# Insert a new node
BST.insert(14)
assert BST.root.left.right.right.value == 14
print('\nInsert 14:')
DF_tree_traversal(BST, order='preorder')

# Minimum of tree
assert BST.minimum().value == 7
print('\nMinimum is: ', BST.minimum().value)

# Find the successor
assert BST.successor(15).value == 16
print('\nSuccessor of 15 is: ', BST.successor(15).value)

# Delete the node
BST.delete(9)
assert BST.root.left.value == 13
print('\nDelete node 9:')
DF_tree_traversal(BST, order='preorder')

Original tree:
-15
|-9
||-7
|||-8
||-13
|-18
||-16
||-19

Insert 14:
-15
|-9
||-7
|||-8
||-13
|||-14
|-18
||-16
||-19

Minimum is:  7

Successor of 15 is:  16

Delete node 9:
-15
|-13
||-7
|||-8
||-14
|-18
||-16
||-19


In [78]:
def BFS_travesal(BT):
    '''Inplementation of Depth-first tree traversal
    
    Args:
    - Binary_Tree: binary tree
    
    Output:
    - BFS_result: list, result of BFS
    '''
    
    BFS_result = []
    current_nodes = [BT.root]
    bottom = False

    while bottom is False:
        val = []
        next_nodes = []
        bottom = True
        for node in current_nodes:
            if node is not None:
                bottom = False
                val.append(node.value)
                next_nodes.append(node.left)
                next_nodes.append(node.right) 
            else:
                val.append(None)  
                next_nodes.append(None)
                next_nodes.append(None) 
              
        BFS_result.append(val)
        current_nodes = next_nodes
    
    return BFS_result[:-1]

In [79]:
def draw_tree(array):
    BST = Bi_Search_Tree()
    for i in array:
        BST.insert(i)

    for vals in BFS_travesal(BST):
        print(vals)

In [80]:
array = [5,8,3,9,2,6,1,4]
draw_tree(array)

[5]
[3, 8]
[2, 4, 6, 9]
[1, None, None, None, None, None, None, None]
